<a href="https://colab.research.google.com/github/hardayal/EVA/blob/master/Session_16/EVA_Assignment_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

###  Model architectured.
###  Defined tf records class.

In [1]:
import tensorflow as tf

In [2]:
print(tf.__version__)

1.15.0


In [0]:
from tensorflow.keras.datasets import cifar10
import sys
from tensorflow.keras import layers

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 6s 0us/step


In [0]:
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

In [0]:
class TFRecords:   

  def _int64_feature(self, value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

  def _bytes_feature(self, value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
    
  def createDataRecord(self, out_filename, images, labels):

    writer = tf.io.TFRecordWriter(out_filename)

    for i in range(len(images)):
      feature = {
          'image_raw': self._bytes_feature(images[i].tostring()),
          'label': self._int64_feature(labels[i])
      }

      example = tf.train.Example(features=tf.train.Features(feature=feature))

      writer.write(example.SerializeToString())

    writer.close()
    sys.stdout.flush()

In [0]:
# Instantite the class

tfRecords = TFRecords()

In [0]:
tfRecords.createDataRecord('train.tfrecords', x_train, y_train)

In [0]:
tfRecords.createDataRecord('test.tfrecords', x_test, y_test)

In [10]:
!ls

sample_data  test.tfrecords  train.tfrecords


In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 5
num_filter = 22
compression = 0.5
dropout_rate = 0.2

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 22, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(5):
        BatchNorm = layers.BatchNormalization()(temp)
        relu = layers.Activation('relu')(BatchNorm)
        Conv2D_3_3 = layers.Conv2D(int(num_filter), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = layers.Dropout(dropout_rate)(Conv2D_3_3)
        concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 22, dropout_rate = 0.2):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = layers.Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = layers.Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    AvgPooling = layers.AveragePooling2D(pool_size=(2,2))(relu)
    flat = layers.Flatten()(AvgPooling)
    output = layers.Dense(num_classes, activation='softmax')(flat)
    
    return output

In [15]:
num_filter = 22
dropout_rate = 0.2
l = 5
input = layers.Input(shape=(img_height, img_width, channel,))
First_Conv2D = layers.Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [16]:
model = tf.keras.Model(inputs=[input], outputs=[output])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 22)   594         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 22)   88          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 22)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [0]:
model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [18]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 22)   594         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 22)   88          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 22)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [0]:
def parser(record):
  keys_to_features = {
      'image_raw': tf.FixedLenFeature((), tf.string),
      'label': tf.FixedLenFeature((), tf.int64)
  }
  parsed = tf.parse_single_example(record, keys_to_features)
  image = tf.decode_raw(parsed['image_raw'], tf.uint8)
  image = tf.cast(image, tf.int32)
  image = tf.reshape(image, [32,32,3])
  label = tf.cast(parsed["label"], tf.int32)
  label = tf.one_hot(label, 10)
  return image, label

In [0]:
def get_dataset_from_TFRecord(filename):
  dataset = tf.data.TFRecordDataset(filenames='train.tfrecords')
  dataset = dataset.map(parser)
  return dataset    

In [21]:
dataset = get_dataset_from_TFRecord('train.tfrecords')
dataset = dataset.batch(512)
dataset = dataset.repeat()

In [0]:
val_dataset = get_dataset_from_TFRecord('test.tfrecords')

In [0]:
val_dataset = val_dataset.batch(512).repeat()

In [24]:
model.fit(dataset, epochs=50, steps_per_epoch=30,
          validation_data=val_dataset,validation_steps=3)

Train on 30 steps, validate on 3 steps
Epoch 1/50
30/30 [==============================] - 12s 413ms/step - loss: 2.0002 - acc: 0.2430 - val_loss: 2.6887 - val_acc: 0.1035
Epoch 2/50
30/30 [==============================] - 5s 164ms/step - loss: 1.6639 - acc: 0.3722 - val_loss: 1.9640 - val_acc: 0.2806
Epoch 3/50
30/30 [==============================] - 5s 165ms/step - loss: 1.5278 - acc: 0.4326 - val_loss: 2.2660 - val_acc: 0.2018
Epoch 4/50
30/30 [==============================] - 6s 188ms/step - loss: 1.4143 - acc: 0.4795 - val_loss: 1.5411 - val_acc: 0.4434
Epoch 5/50
30/30 [==============================] - 5s 165ms/step - loss: 1.3139 - acc: 0.5169 - val_loss: 1.8441 - val_acc: 0.3809
Epoch 6/50
30/30 [==============================] - 5s 165ms/step - loss: 1.2692 - acc: 0.5396 - val_loss: 1.4352 - val_acc: 0.4674
Epoch 7/50
30/30 [==============================] - 5s 165ms/step - loss: 1.1992 - acc: 0.5652 - val_loss: 1.3335 - val_acc: 0.5208
Epoch 8/50
30/30 [==================

In [25]:
model.fit(dataset, epochs=50, steps_per_epoch=30,
          validation_data=val_dataset,validation_steps=3)

Train on 30 steps, validate on 3 steps
Epoch 1/50
30/30 [==============================] - 8s 267ms/step - loss: 0.5678 - acc: 0.7988 - val_loss: 0.6898 - val_acc: 0.7734
Epoch 2/50
30/30 [==============================] - 5s 164ms/step - loss: 0.5918 - acc: 0.7908 - val_loss: 0.7778 - val_acc: 0.7591
Epoch 3/50
30/30 [==============================] - 5s 165ms/step - loss: 0.6128 - acc: 0.7856 - val_loss: 0.7725 - val_acc: 0.7539
Epoch 4/50
30/30 [==============================] - 5s 164ms/step - loss: 0.5723 - acc: 0.7987 - val_loss: 0.5842 - val_acc: 0.7936
Epoch 5/50
30/30 [==============================] - 5s 164ms/step - loss: 0.5889 - acc: 0.7926 - val_loss: 1.0822 - val_acc: 0.6973
Epoch 6/50
30/30 [==============================] - 5s 164ms/step - loss: 0.5911 - acc: 0.7951 - val_loss: 1.2491 - val_acc: 0.6504
Epoch 7/50
30/30 [==============================] - 5s 165ms/step - loss: 0.5692 - acc: 0.7996 - val_loss: 0.7213 - val_acc: 0.7741
Epoch 8/50
30/30 [===================